In [1]:

from collections import defaultdict, OrderedDict, deque
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_ind, shapiro, f_oneway, mannwhitneyu

from sklearn import metrics
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from statsmodels.stats.proportion import proportions_ztest
from importlib import reload
import logging
import os

%matplotlib inline
plt.style.use('seaborn-v0_8')


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# Finding outliers by Tukey method with adjustment option for iqr boundaries.

def outliers_iqr(data, feature, left_iqr=1.5, right_iqr=1.5, log_scale=False):
    """Function for finding outliers by Tukey method with adjustment option for iqr's multiplier.

    Args:
        data (DataFrame): DataFrame which will be used to find outliers.
        feature (string): Name of a column in DF which will be inspected for outliers.
        left_iqr (float, optional): lower boundary multiplier. Defaults to 1.5.
        right_iqr (float, optional): upper boundary multiplier. Defaults to 1.5.
        log_scale (bool, optional): converting data in logarithmic representation in case of lognormal destribution of
        original data. Defaults to False.

    Returns:
        DataFrame: returns two copies of the original DataFrame contain DF's with outliers and cleaned data.
    """    ''''''

    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]

    quartile1, quartile3 = x.quantile(0.25), x.quantile(0.75)

    iqr = quartile3 - quartile1

    lower_bound = quartile1 - (iqr*left_iqr)

    upper_bound = quartile3 + (iqr*right_iqr)

    outliers = data[(x < lower_bound) | (x > upper_bound)]

    cleaned = data[(x > lower_bound) & (x < upper_bound)]

    print(f'Number of outliers by Tukey\'s method: {outliers.shape[0]}')
    print(f'Resulting number of lines cleared of outliers: {cleaned.shape[0]}')

    return outliers, cleaned


In [3]:
# Finding outliers by z-method with adjustment option for boundaries.

def outliers_z_score(data, feature, left_mod=3, right_mod=3, log_scale=False):
    """Function for finding outliers by z-method with adjustment option for left and right multiplier.

    Args:
        data (DataFrame): DataFrame which will be used to find outliers.
        feature (string): Name of a column in DF which will be inspected for outliers.
        left_mod (int, optional): lower boundary multiplier. Defaults to 3.
        right_mod (int, optional): upper boundary multiplier. Defaults to 3.
        log_scale (bool, optional): converting data in logarithmic representation in case of lognormal destribution of
        original data. Defaults to False.

    Returns:
        DataFrame: returns two copies of the original DataFrame contain DF's with outliers and cleaned data.
    """    ''''''

    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]

    mu = x.mean()

    sigma = x.std()

    lower_bound = mu - left_mod * sigma

    upper_bound = mu + right_mod * sigma

    outliers = data[(x < lower_bound) | (x > upper_bound)]

    cleaned = data[(x > lower_bound) & (x < upper_bound)]

    print(f'Number of outliers by z-method: {outliers.shape[0]}')
    print(f'Resulting number of lines cleared of outliers: {cleaned.shape[0]}')

    return outliers, cleaned


In [4]:
# Duplicate finding function in lines

def dupl_data_remove(data, immune_col=None):
    """Function for finding full dupliacates in lines

    Args:
        data (DataFrame): DataFrame which will be used to find duplicates.
        immune_col (str or tupple, optional): name of the column/s which the function will pass. Defaults to None.

    Returns:
        DataFrame: returns the copy of the original DataFrame cleaned from full duplicates in line.
    """    ''''''

    if immune_col is None:
        dupl_columns = list(data.columns)

    else:
        dupl_columns = list(data.columns)
        try:
            for col in immune_col:
                dupl_columns.remove(immune_col)
        except ValueError:
            pass

    mask = data.duplicated(subset=dupl_columns)

    data_duplicates = data[mask]
    print(f'Number of duplicates: {data_duplicates.shape[0]}')

    data_dedupped = data.drop_duplicates(subset=dupl_columns)
    print(
        f'Resulting number of lines cleared of duplicates: {data_dedupped.shape[0]}')

    return data_dedupped


In [5]:
# Duplicate finding function in lines (old version)

def dupl_data_remove(data, immune_col=None):
    """Function for finding full dupliacates in lines

    Args:
        data (DataFrame): DataFrame which will be used to find duplicates.
        immune_col (str or tupple, optional): name of the column/s which the function will pass. Defaults to None.

    Returns:
        DataFrame: returns the copy of the original DataFrame cleaned from full duplicates in line.
    """    ''''''

    if immune_col is None:
        dupl_columns = list(data.columns)
    else:
        dupl_columns = list(data.columns)
        dupl_columns.remove(immune_col)

    mask = data.duplicated(subset=dupl_columns)

    data_duplicates = data[mask]
    print(f'Number of duplicates: {data_duplicates.shape[0]}')

    data_dedupped = data.drop_duplicates(subset=dupl_columns)
    print(
        f'Resulting number of lines cleared of duplicates: {data_dedupped.shape[0]}')

    return data_dedupped


In [6]:
# Функция по поиску неинформативных признаков

def low_info_col_drop(data, top_freq_thresh=0.95, nuniq_thresh=0.95):
    """_summary_

    Args:
        data (_type_): _description_
        top_freq_thresh (float, optional): _description_. Defaults to 0.95.
        nuniq_thresh (float, optional): _description_. Defaults to 0.95.

    Returns:
        _type_: _description_
    """    ''''''

    low_info_col_list = []                                                            #

    for col in data.columns:                                                         #
        top_freq = data[col].value_counts(
            normalize=True).max()                      #
        nunique_ratio = data[col].nunique(
        ) / data[col].count()                      #

    if top_freq > top_freq_thresh:                                                   #
        #
        low_info_col_list.append(col)
        #
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')

    if nunique_ratio > nuniq_thresh:                                                 #
        #
        low_info_col_list.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')          #

    return low_info_col_list


In [7]:
# Function for cleaning data frame from null data features by conditional level of null data.

def null_data_col_drop(data, null_thresh=0.4, immune_col=None):
    """Function for cleaning the data of empty data features.

    Args:
        data (DataFrame): DataFrame which will be used for cleaning.
        null_thresh (float, optional): minimal threshold for removing the feature. Defaults to 0.4.
        immune_col (str or tupple, optional): name of the column/s which the function will pass. Defaults to None.

    Returns:
        DataFrame: returns the copy of the original DataFrame cleaned from null data columns.
    """    ''''''

    # creating the temp frame with replased 0 by NaN
    temp_data = data.replace({0: np.nan})
    null_data_col_list = []  # empty list for future drop of columns

    for col in temp_data.columns:
        try:
            col_null = round(temp_data[col].isnull().value_counts(normalize=True), 2)[
                True]  # percent of NaN in every column
        except KeyError:  # if there is no NaN
            col_null = 0

        if col_null > null_thresh:  # comparison with acceptable level of empty data in column
            null_data_col_list.append(col)
            print(f'{col}: {col_null*100}% zero values')

    if immune_col is None:  # checkin' is there any column/s that the func should skip in drop process
        pass
    else:
        try:
            for col in immune_col:
                null_data_col_list.remove(immune_col)
        except ValueError:
            pass

    drop_data = data.drop(null_data_col_list, axis=1)
    print(
        f'{drop_data.shape[1]} features with less than {null_thresh*100}% null data')

    return drop_data


In [8]:
#One hot encoding method on example
df = pd.DataFrame({
    'Name':['Joe', 'Sue', 'Kirk'],
    'Dish':['Shit','Cake','Shit, Cake']
})
df['Dish_new'] = df['Dish'].str.replace(' ','')
df_encoding = df['Dish_new'].str.get_dummies(',')
df_final = pd.concat([df, df_encoding], axis=1)
df_final.drop(['Dish', 'Dish_new'], axis=1, inplace=True)

In [9]:
# Доверительные интервалы при известном истинном стандартном отклонении
def confidence_interval_sigma(n, x_mean, sigma, gamma=0.95):
    """Function to evaluate confidence interval with known true standard deviation

    Args:
        n (int): sample size
        x_mean (int): sample mean
        sigma (int): true standard deviation
        gamma (float): reliability level 0-1

    Returns:
        float: lower_bound, upper_bound
    """
    alpha = 1 - gamma  # уровень значимости

    z_crit = round(-norm.ppf(alpha/2), 2)  # z критическое

    eps = z_crit * sigma/(n ** 0.5)  # погрешность
    lower_bound = round(x_mean - eps, 2)  # левая (нижняя) граница
    upper_bound = round(x_mean + eps, 2)  # правая (верхняя) граница
    # создаём кортеж из округлённых границ интервала
    confidence_interval = (lower_bound, upper_bound)
    # выводим результат
    print(
        f'Доверительный интервал: {lower_bound, upper_bound}, z-крит: {z_crit}')
    return lower_bound, upper_bound, z_crit


In [10]:
# Доверительные интервалы при неизвестном истинном стандартном отклонении
def confidence_interval(n, x_mean, x_std, gamma=0.95):
    """_summary_

    Args:
        n (int): sample size
        x_mean (int): sample mean
        x_std (int, float): sample standard deviation
        gamma (float): reliability level 0-1
        
    """
    k = n - 1  # число степеней свободы
    alpha = 1 - gamma  # уровень значимости

    t_crit = -t.ppf(alpha/2, k)  # t-критическое

    eps = t_crit * x_std/(n ** 0.5)  # погрешность
    lower_bound = round(x_mean - eps, 2)  # левая (нижняя) граница
    upper_bound = round(x_mean + eps, 2)  # правая (верхняя) граница
    # создаём кортеж из округлённых границ интервала
    confidence_interval = (lower_bound, upper_bound)
    #print(
    #    f'Доверительный интервал: {lower_bound, upper_bound}, t-крит: {t_crit}')
    return lower_bound, upper_bound


In [11]:
# Доверительный интервал для пропорции (например конверсии)
def proportions_conf_interval(n, x_p, gamma=0.95):
    """Function to evaluate confidence interval for proportion and
    true standard deviation.

    Args:
        n (int): sample size
        x_p (int): sample proportion (% of success)
        gamma (float): reliability level 0-1

    Returns:
        float: lower_bound, upper_bound, sigma
    """
    alpha = 1 - gamma  # уровень значимости
    z_crit = -norm.ppf(alpha/2)  # z критическое
    eps = z_crit * (x_p * (1 - x_p) / n) ** 0.5  # погрешность
    sigma = round((x_p * (1 - x_p))**0.5, 3) #true standard deviation

    lower_bound = round((x_p - eps)*100, 2)  # левая (нижняя) граница
    upper_bound = round((x_p + eps)*100, 2)  # правая (верхняя) граница

    #print(
    #    f'Доверительный интервал: {lower_bound, upper_bound}, z-крит: {z_crit}, sigma: {sigma}')
    return lower_bound, upper_bound, sigma


In [12]:
# Доверительный интервал разницы пропорции (например конверсии)
def diff_proportions_conf_interval(n, x_p, gamma=0.95):
    """Function to evaluate confidence interval for delta of proportion.

    Args:
        n (list): list of samples sizes (for example: n = [a_data['user_id'].count(), b_data['user_id'].count()])
        x_p (list): list of mean samples proportions (for example: xp = [a_data['converted'].mean(), b_data['converted'].mean()])
        gamma (float, optional): reliability level 0-1

    Returns:
        float: lower_bound, upper_bound (%)
    """

    alpha = 1 - gamma  # уровень значимости
    diff = x_p[1] - x_p[0]  # выборочная разн~ица конверсий групп B и A
    z_crit = -norm.ppf(alpha/2)  # z критическое
    eps = z_crit * (x_p[0] * (1 - x_p[0])/n[0] + x_p[1] *
                    (1 - x_p[1])/n[1]) ** 0.5  # погрешность

    lower_bound = round((diff - eps)*100, 2)  # левая (нижняя) граница
    upper_bound = round((diff + eps)*100, 2)  # правая (верхняя) граница
    # возвращаем кортеж из округлённых границ интервала
    return lower_bound, upper_bound


In [13]:
# Функция для создания лог-файла и записи в него информации
def get_logger(path, file):
    """[Создает лог-файл для логирования в него]
    Аргументы:
        path {string} -- путь к директории
        file {string} -- имя файла
    Возвращает:
        [obj] -- [логер]
    """
    # проверяем, существует ли файл
    log_file = os.path.join(path, file)

    # если  файла нет, создаем его
    if not os.path.isfile(log_file):
        open(log_file, "w+").close()

    # поменяем формат логирования
    file_logging_format = "%(levelname)s: %(asctime)s: %(message)s"

    # конфигурируем лог-файл
    logging.basicConfig(level=logging.INFO,
                        format=file_logging_format)
    logger = logging.getLogger()

    # создадим хэнлдер для записи лога в файл
    handler = logging.FileHandler(log_file)

    # установим уровень логирования
    handler.setLevel(logging.INFO)

    # создадим формат логирования, используя file_logging_format
    formatter = logging.Formatter(file_logging_format)
    handler.setFormatter(formatter)

    # добавим хэндлер лог-файлу
    logger.addHandler(handler)
    return logger


In [14]:
# Независимый T-тест
def t_test(data, feature_1='', feature_2=''):
    """T-test

    Args:
        data (DataFrame): DataFrame
        feature_1 (str, optional): Fearure_1.
        feature_2 (str, optional): Fearure_2.

    Returns:
        statistic, p_value
    """
    test_results = ttest_ind(data[feature_1], data[feature_2], equal_var=True)

    statistic = round(test_results[0], 4)
    p_value = round(test_results[1], 4)

    return statistic, p_value


In [15]:
# Metric function print
def print_metrics(y_train, y_train_predict, y_test, y_test_predict):
    print(f'Train R^2: {round(metrics.r2_score(y_train, y_train_predict),3)}')
    print(f'Train MAE: {round(metrics.mean_absolute_error(y_train, y_train_predict),3)}')
    print(f'Train MAPE: {round(metrics.mean_absolute_percentage_error(y_train, y_train_predict)*100,2)}')
    
    print('\n')
    
    print(f'Test R^2: {round(metrics.r2_score(y_test, y_test_predict),3)}')
    print(f'Test MAE: {round(metrics.mean_absolute_error(y_test, y_test_predict),3)}')
    print(f'Test MAPE: {round(metrics.mean_absolute_percentage_error(y_test, y_test_predict)*100,2)}')

---

### Методы

In [16]:
#Создаём список из 20 возможных значений от 0.001 до 1 L1- регуляризация
alpha_list = np.linspace(0.001, 1, 20)
#Создаём пустые списки, в которые будем добавлять результаты 
train_scores = []
test_scores = []
for alpha in alpha_list:
    #Создаём объект класса линейной регрессии с L1-регуляризацией
    lasso_lr_poly = linear_model.Lasso(alpha=alpha, max_iter=10000)
    #Обучаем модель
    lasso_lr_poly.fit(X_train_scaled_poly, y_train)
    #Делаем предсказание для тренировочной выборки
    y_train_predict_poly = lasso_lr_poly.predict(X_train_scaled_poly)
    #Делаем предсказание для тестовой выборки
    y_test_predict_poly = lasso_lr_poly.predict(X_test_scaled_poly)
    #Рассчитываем коэффициенты детерминации для двух выборок и добавляем их в списки
    train_scores.append(metrics.r2_score(y_train, y_train_predict_poly))
    test_scores.append(metrics.r2_score(y_test, y_test_predict_poly))

NameError: name 'X_train_scaled_poly' is not defined

In [ ]:
# Визуализируем изменение R^2 в зависимости от alpha
fig, ax = plt.subplots(figsize=(12, 4))  # фигура + координатная плоскость
# линейный график для тренировочной выборки
ax.plot(alpha_list, train_scores, label='Train')
# линейный график для тестовой выборки
ax.plot(alpha_list, test_scores, label='Test')
ax.set_xlabel('Alpha')  # название оси абсцисс
ax.set_ylabel('R^2')  # название оси ординат
ax.set_xticks(alpha_list)  # метки по оси абсцисс
ax.xaxis.set_tick_params(rotation=45)  # поворот меток на оси абсцисс
ax.legend()  # отображение легенды

In [ ]:
# Визуализируем ошибки
fig, axes = plt.subplots(figsize=(12,6))
# Ошибки модели на тренировочной выборке
y_train_errors = y_train - y_train_pred
# Ошибки модели на тестовой выборке
y_test_errors = y_test - y_test_pred
# Создаем df из ошибок
predict_df = pd.DataFrame({
    'Train_errors': y_train_errors,
    'Test_errors': y_test_errors
})
# Строим boxplot для ошибок
sns.boxplot(data=predict_df, ax= axes)
ax.set_xlabel('Model errors')
ax.set_ylabel('Model')


---

In [ ]:
#Создаём список из 20 возможных значений от 0.001 до 1 L2-регуляризация
alpha_list = np.linspace(0.01, 10, 20)
#Создаём пустые списки, в которые будем добавлять результаты 
train_scores = []
test_scores = []
for alpha in alpha_list:
    #Создаём объект класса линейной регрессии с L1-регуляризацией
    lasso_lr_poly = linear_model.Ridge(alpha=alpha, max_iter=10000)
    #Обучаем модель
    lasso_lr_poly.fit(X_train_scaled_poly, y_train)
    #Делаем предсказание для тренировочной выборки
    y_train_predict_poly = lasso_lr_poly.predict(X_train_scaled_poly)
    #Делаем предсказание для тестовой выборки
    y_test_predict_poly = lasso_lr_poly.predict(X_test_scaled_poly)
    #Рассчитываем коэффициенты детерминации для двух выборок и добавляем их в списки
    train_scores.append(metrics.r2_score(y_train, y_train_predict_poly))
    test_scores.append(metrics.r2_score(y_test, y_test_predict_poly))

In [ ]:
# Визуализируем изменение R^2 в зависимости от alpha
fig, ax = plt.subplots(figsize=(12, 4))  # фигура + координатная плоскость
# линейный график для тренировочной выборки
ax.plot(alpha_list, train_scores, label='Train')
# линейный график для тестовой выборки
ax.plot(alpha_list, test_scores, label='Test')
ax.set_xlabel('Alpha')  # название оси абсцисс
ax.set_ylabel('R^2')  # название оси ординат
ax.set_xticks(alpha_list)  # метки по оси абсцисс
ax.xaxis.set_tick_params(rotation=45)  # поворот меток на оси абсцисс
ax.legend()  # отображение легенды

In [ ]:
# Визуализируем ошибки
fig, axes = plt.subplots(figsize=(12,6))
# Ошибки модели на тренировочной выборке
y_train_errors = y_train - y_train_pred
# Ошибки модели на тестовой выборке
y_test_errors = y_test - y_test_pred
# Создаем df из ошибок
predict_df = pd.DataFrame({
    'Train_errors': y_train_errors,
    'Test_errors': y_test_errors
})
# Строим boxplot для ошибок
sns.boxplot(data=predict_df, ax= axes)
ax.set_xlabel('Model errors')
ax.set_ylabel('Model')


---

In [ ]:
#Функция для визуализации модели
def plot_probabilities_2d(X, y, model):
    #Генерируем координатную сетку из всех возможных значений для признаков
    #Glucose изменяется от x1_min = 44 до x2_max = 199, 
    #BMI — от x2_min = 18.2 до x2_max = 67.1
    #Результат работы функции — два массива xx1 и xx2, которые образуют координатную сетку
    xx1, xx2 = np.meshgrid(
        np.arange(X.iloc[:, 0].min()-1, X.iloc[:, 0].max()+1, 0.1),
        np.arange(X.iloc[:, 1].min()-1, X.iloc[:, 1].max()+1, 0.1)
    )
    #Вытягиваем каждый из массивов в вектор-столбец — reshape(-1, 1)
    #Объединяем два столбца в таблицу с помощью hstack
    X_net = np.hstack([xx1.reshape(-1, 1), xx2.reshape(-1, 1)])
    #Предсказываем вероятность для всех точек на координатной сетке
    #Нам нужна только вероятность класса 1
    probs = model.predict_proba(X_net)[:, 1]
    #Переводим столбец из вероятностей в размер координатной сетки
    probs = probs.reshape(xx1.shape)
    #Создаём фигуру и координатную плоскость
    fig, ax = plt.subplots(figsize = (10, 5))
    #Рисуем тепловую карту вероятностей
    contour = ax.contourf(xx1, xx2, probs, 100, cmap='bwr')
    #Рисуем разделяющую плоскость — линию, где вероятность равна 0.5
    bound = ax.contour(xx1, xx2, probs, [0.5], linewidths=2, colors='black');
    #Добавляем цветовую панель 
    colorbar = fig.colorbar(contour)
    #Накладываем поверх тепловой карты диаграмму рассеяния
    sns.scatterplot(x=X.iloc[:, 0], y=X.iloc[:, 1], hue=y, palette='seismic', ax=ax)
    #Даём графику название
    ax.set_title('Scatter Plot with Decision Boundary');
    #Смещаем легенду в верхний левый угол вне графика
    ax.legend(bbox_to_anchor=(-0.05, 1))

In [ ]:
# функция, которая принимает количество кластеров для k-means и матрицу с признаками объектов и возвращает инерцию 
def get_inertia(cluster_num, X):
# инициализируем алгоритм кластеризации
    k_means =  KMeans(n_clusters=cluster_num, random_state=42)
# запускаем алгоритм k-means
    k_means.fit(X)
# находим значение инерции
    inertia = k_means.inertia_
# возвращаем значение инерции
    return inertia

In [ ]:
# напишем функцию, как и при подсчете метода локтя
def get_silhouette(cluster_num, X):
    k_means =  KMeans(n_clusters=cluster_num, init='k-means++', n_init=10, random_state=42)
    k_means.fit(X)
# подсчитаем метрику силуэта, передав данные и то, к каким кластерам относятся объекты
    silhouette = silhouette_score(X, k_means.predict(X))
    return silhouette

In [ ]:
def plot_learning_curve(model, X, y, cv, scoring="f1", ax=None, title=""):
    # Вычисляем координаты для построения кривой обучения
    train_sizes, train_scores, valid_scores = model_selection.learning_curve(
        estimator=model,  # модель
        X=X,  # матрица наблюдений X
        y=y,  # вектор ответов y
        cv=cv,  # кросс-валидатор
        scoring=scoring,  # метрика
    )
    # Вычисляем среднее значение по фолдам для каждого набора данных
    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)
    # Если координатной плоскости не было передано, создаём новую
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 4))  # фигура + координатная плоскость
    # Строим кривую обучения по метрикам на тренировочных фолдах
    ax.plot(train_sizes, train_scores_mean, label="Train")
    # Строим кривую обучения по метрикам на валидационных фолдах
    ax.plot(train_sizes, valid_scores_mean, label="Valid")
    # Даём название графику и подписи осям
    ax.set_title("Learning curve: {}".format(title))
    ax.set_xlabel("Train data size")
    ax.set_ylabel("Score")
    # Устанавливаем отметки по оси абсцисс
    ax.xaxis.set_ticks(train_sizes)
    # Устанавливаем диапазон оси ординат
    ax.set_ylim(0, 1)
    # Отображаем легенду
    ax.legend()

In [ ]:
def plot_cluster_profile(grouped_data, n_clusters):
    # Нормализуем сгруппированные данные, приводя их к масштабу 0-1.
    scaler = preprocessing.MinMaxScaler()
    grouped_data = pd.DataFrame(scaler.fit_transform(grouped_data), columns=grouped_data.columns)
    # Создаем список признаков
    features = grouped_data.columns
    # Создаем пустую фигуру
    fig = go.Figure()
    # В цикле визуализируем полярную диаграмму для каждого кластера
    for i in range(n_clusters):
        # Создаем полярную диаграмму и добавляем ее на общий график
        fig.add_trace(go.Scatterpolar(
            r=grouped_data.iloc[i].values, # радиусы
            theta=features, # название засечек
            fill='toself', # заливка многоугольника цветом
            name=f'Cluster {i}', # название - номер кластера
        ))
    # Обновляем параметры фигуры
    fig.update_layout(
        showlegend=True, # отображение легенды
        autosize=False, # устаналиваем свои размеры графика
        width=800, # ширина (в пикселях)
        height=800, # высота (в пикселях)
    )
    # Отображаем фигуру
    fig.show()
    

In [ ]:
#formating date from raw json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date

In [ ]:
# Поиск максимальной клики для применения в отборе признаков на основе корреляционной матрицы

# Каждая из вершин графа представляет собой отдельный признак,
# Ребром соединяются только те вершины, которые не являются сильно коррелированными
# при заданном пороге корреляции.

class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.adj = [[] for _ in range(vertices)]

    def add_edge(self, u, v):
        self.adj[u].append(v)
        self.adj[v].append(u)

    def bron_kerbosch(self, r, p, x):
        if len(p) == 0 and len(x) == 0:
            print(r)
            return r

        for v in p[:]:
            self.bron_kerbosch(
                r + [v], [vertex for vertex in p if vertex in self.adj[v]], 
                [vertex for vertex in x if vertex in self.adj[v]])
            p.remove(v)
            x.append(v)

    def find_cliques(self):
        self.bron_kerbosch([], list(range(self.V)), [])


# Пример использования





g = Graph(5)
g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(2, 3)
g.add_edge(3, 4)

g.find_cliques()


[0, 1, 2]
[1, 2, 3]
[3, 4]


In [ ]:
# Метод отжига


import random
import math

def simulated_annealing(initial_state):
    """Peforms simulated annealing to find a solution"""
    initial_temp = 90
    final_temp = .1
    alpha = 0.01

    current_temp = initial_temp

    # Start by initializing the current state with the initial state
    current_state = initial_state
    solution = current_state

    while current_temp > final_temp:
        neighbor = random.choice(get_neighbors())

        # Check if neighbor is best so far
        cost_diff = get_cost(self.current_state) = get_cost(neighbor)

        # if the new solution is better, accept it
        if cost_diff > 0:
            solution = neighbor
        # if the new solution is not better, accept it with a probability of e^(-cost/temp)
        else:
            if random.uniform(0, 1) < math.exp(cost_diff / current_temp):
                solution = neighbor
        # decrement the temperature
        current_temp -= alpha

    return solution

def get_cost(state):
    """Calculates cost of the argument state for your solution."""
    raise NotImplementedError

def get_neighbors(state):
    """Returns neighbors of the argument state for your solution."""
    raise NotImplementedError

In [ ]:
# Moving average prediction
def ma_prediction(train_data, test_data, window_size=3, pred_num=3):
    '''Simple function to predict the moving average of a series.'''
    data_ma = train_data.rolling(window_size).mean()
    last_values = data_ma.tail(window_size).values
    prediction = np.array([last_values[-1] +
                           (last_values[-1] - last_values[-2]) * i for i in range(1, pred_num + 1)])
    prediction = pd.DataFrame(
        prediction, columns=test_data.columns, index=test_data.index)
    return data_ma, prediction


In [ ]:
# Qini-score | Qini-curve
def qini_df(df, title='train', figsize=(5, 3)):
    # Отранжируем выборку по значению uplift в убывающем порядке
    ranked = df.sort_values("uplift_score", ascending=False)
    
    N_c = sum(ranked['target_class'] <= 1)
    N_t = sum(ranked['target_class'] >= 2)
    
    # Посчитаем в отсортированном датафрейме основные показатели, которые используются при расчете qini
    ranked['n_c1'] = 0
    ranked['n_t1'] = 0
    ranked.loc[ranked.target_class == 1,'n_c1'] = 1
    ranked.loc[ranked.target_class == 3,'n_t1'] = 1
    ranked['n_c1/nc'] = ranked.n_c1.cumsum() / N_c
    ranked['n_t1/nt'] = ranked.n_t1.cumsum() / N_t
    
    # Посчитаем qini curve и рандомную прямую под ней
    ranked['uplift'] = round(ranked['n_t1/nt'] - ranked['n_c1/nc'],5)
    # Добавим случайную кривую
    ranked['random_uplift'] = round(ranked["uplift_score"].rank(pct=True, ascending=False) * ranked['uplift'].iloc[-1],5)
    
    ranked["n"] = ranked["uplift_score"].rank(pct=True, ascending=False)
    
    # Немного кода для визуализации
    fig = plt.figure(figsize=figsize)
    plt.plot(ranked['n'], ranked['uplift'], color='r', label='Model')
    plt.plot(ranked['n'], ranked['random_uplift'], color='b', label='RandomModel')
    plt.legend()
    plt.title('Qini-curve for {} samples'.format(title))
    plt.show()
    quni_score = (ranked['uplift'] - ranked['random_uplift']).sum()
    print('Qini score: {:.3f}'.format(quni_score))